### **Script para criação do arquivo chave.txt**  

usado para lançamento de cte >Especificos Mafra>Rotinas>Fretes

In [11]:
# SCRIPT IN PYTHON BY DINO GREJO - DINOGREJO@GMAIL.COM GITHUB: DGREJ

import os
import re

# Pasta contendo os arquivos XML
caminho_pasta = r'C:\Users\dino.grejo\OneDrive - VIVEO\Documentos\python\xml'

# Caminho para salvar o arquivo chave.txt
caminho_arquivo_chave = r'C:\Users\dino.grejo\OneDrive - VIVEO\Documentos\python\chave.txt'

# Regex para extrair os números da tag chave
regex = r'<chCTe>(\d+)</chCTe>'

# Abrindo o arquivo de texto para escrita
with open(caminho_arquivo_chave, 'w') as arquivo_chave:
    # Escrevendo "CHAVE" no cabeçalho do arquivo TXT
    arquivo_chave.write('CHAVE\n')
    
    # Iterando sobre cada arquivo na pasta
    for nome_arquivo in os.listdir(caminho_pasta):
        if nome_arquivo.endswith('.xml'):
            # Abrindo o arquivo XML e lendo o conteúdo
            with open(os.path.join(caminho_pasta, nome_arquivo), 'r') as arquivo_xml:
                conteudo = arquivo_xml.read()
                # Extraindo os números da tag chave usando regex
                match = re.search(regex, conteudo)
                if match:
                    numero_chave = match.group(1)
                    # Escrevendo o número da chave no arquivo de texto
                    arquivo_chave.write(numero_chave + ",1" + '\n')

print(f"Arquivo chave.txt gerado em: {caminho_arquivo_chave}")


Arquivo chave.txt gerado em: C:\Users\dino.grejo\OneDrive - VIVEO\Documentos\python\chave.txt


### **Script para criação de planilha para lançamento de CT-e CIF**  
usado para lançamento de cte >Especificos Mafra>Rotinas>Fretes

In [12]:
#SCRIPT IN PYTHON BY DINO GREJO - DINOGREJO@GMAIL.COM GITHUB: DGREJ

import os
import xml.etree.ElementTree as ET
import pandas as pd

# Pasta contendo os arquivos XML
caminho_pasta = r'C:\Users\dino.grejo\OneDrive - VIVEO\Documentos\python\xml'

# Lista para armazenar os dados extraídos
dados = []

# Função para extrair as informações de cada XML
def extrair_dados_do_xml(caminho_xml):
    tree = ET.parse(caminho_xml)
    root = tree.getroot()
    
    # Definir o namespace
    ns = {'nfe': 'http://www.portalfiscal.inf.br/cte'}
    
    # Função auxiliar para extrair texto de elementos com verificação
    def get_element_text(path):
        element = root.find(path, ns)
        return element.text if element is not None else ''
    
    # Ajuste para remover os dois primeiros dígitos
    def ajustar_codigo_municipio(codigo):
        return codigo[2:] if len(codigo) > 2 else codigo

    # Extrair as informações desejadas
    cte_info = {
        'CNPJ REMETENTE': get_element_text('.//nfe:rem/nfe:CNPJ'),
        'REMETENTE': get_element_text('.//nfe:rem/nfe:xNome'),
        'CNPJ TRANSP.':get_element_text('.//nfe:emit/nfe:CNPJ'),
        'TRANSPORTADOR':get_element_text('.//nfe:emit/nfe:xNome'),
        'DESTINATARIO': get_element_text('.//nfe:dest/nfe:xNome'),
        'CNPJ DESTINATARIO': get_element_text('.//nfe:dest/nfe:CNPJ'),
        'CTE': get_element_text('.//nfe:ide/nfe:nCT'),
        'SERIE': get_element_text('.//nfe:ide/nfe:serie'),
        'VALOR': get_element_text('.//nfe:vPrest/nfe:vTPrest').replace('.', ','),
        'CHAVEnf': '\n'.join([chave.text for chave in root.findall('.//nfe:infDoc/nfe:infNFe/nfe:chave', ns)]),
        'EMISSAO': get_element_text('.//nfe:ide/nfe:dhEmi').split('T')[0] if get_element_text('.//nfe:ide/nfe:dhEmi') else '',  # Formato de data apenas
        'UFIni': get_element_text('.//nfe:ide/nfe:UFIni'),
        'cMunIni': ajustar_codigo_municipio(get_element_text('.//nfe:ide/nfe:cMunIni')),
        'xMunIni': get_element_text('.//nfe:ide/nfe:xMunIni'),
        'UFFim': get_element_text('.//nfe:ide/nfe:UFFim'),
        'cMunFim': ajustar_codigo_municipio(get_element_text('.//nfe:ide/nfe:cMunFim')),
        'xMunFim': get_element_text('.//nfe:ide/nfe:xMunFim'),
        'chCTe': get_element_text('.//nfe:infProt/nfe:chCTe'),
        'vTrib': get_element_text('.//nfe:vPrest/nfe:vTPrest').replace('.', ','),
        'icms%': get_element_text('.//nfe:ICMS/nfe:ICMS00/nfe:pICMS'),
        'vICMS': get_element_text('.//nfe:ICMS/nfe:ICMS00/nfe:vICMS').replace('.', ','),
        'ICMSOutUF%': get_element_text('.//nfe:ICMS/nfe:ICMSOutraUF/nfe:pICMSOutraUF'),
        'vICMSOutUF': get_element_text('.//nfe:ICMS/nfe:ICMSOutraUF/nfe:vICMSOutraUF').replace('.', ','),
        
    }

     # Ajuste para a data de emissão no formato desejado (dd/mm/aaaa)
    if cte_info['EMISSAO']:
        data_emissao = cte_info['EMISSAO'].split('-')
        cte_info['EMISSAO'] = f"{data_emissao[2]}/{data_emissao[1]}/{data_emissao[0]}"
    
    return cte_info

# Percorrer todos os arquivos XML na pasta
for arquivo in os.listdir(caminho_pasta):
    if arquivo.endswith('.xml'):
        caminho_xml = os.path.join(caminho_pasta, arquivo)
        dados.append(extrair_dados_do_xml(caminho_xml))

# Criar um DataFrame com os dados extraídos
df = pd.DataFrame(dados)

# Adicionar colunas vazias (em branco) ao DataFrame
colunas_extras = ['Venc.', 'Data lanç.', 'FATURA','serviço', 'COD']
for coluna in colunas_extras:
    df[coluna] = ''  # Preenche a coluna com valores vazios

# Ajustar a ordem das colunas de acordo com o modelo Excel
df = df[['Venc.', 'Data lanç.', 'FATURA','serviço', 'CNPJ REMETENTE', 'REMETENTE', 'CNPJ TRANSP.', 'COD', 'TRANSPORTADOR', 'DESTINATARIO', 'CNPJ DESTINATARIO', 'CTE', 'SERIE', 'VALOR', 'CHAVEnf', 'EMISSAO', 'UFIni', 'cMunIni', 'xMunIni', 'UFFim', 
         'cMunFim', 'xMunFim', 'chCTe', 'vTrib', 'icms%', 'vICMS', 'ICMSOutUF%', 'vICMSOutUF']]

# Gerar a planilha Excel
caminho_excel = r'C:\Users\dino.grejo\OneDrive - VIVEO\Documentos\python\Fatura_CTes_CIF.xlsx'
df.to_excel(caminho_excel, index=False)

print(f'Planilha gerada em: {caminho_excel}')


Planilha gerada em: C:\Users\dino.grejo\OneDrive - VIVEO\Documentos\python\Fatura_CTes_CIF.xlsx


### **CODE AJUSTADO FOB HL**
#### **INSERIR VIA PROMPR VENCIMENTO E FILIAL**

In [ ]:
# SCRIPT IN PYTHON BY DINO GREJO - DINOGREJO@GMAIL.COM GITHUB: DGREJ

import os
import xml.etree.ElementTree as ET
import pandas as pd

# Pasta contendo os arquivos XML
caminho_pasta = r'C:\Users\dino.grejo\OneDrive - VIVEO\Documentos\python\xml'

# Solicitar valores via prompt
vencimento = input("Digite a data de vencimento para a coluna 'Venc.': ")
nome_filial_numero = input("Digite o número para a 'NOME FILIAL' (ex: 001002): ")

# Lista para armazenar os dados extraídos
dados = []

# Função para extrair as informações de cada XML
def extrair_dados_do_xml(caminho_xml):
    tree = ET.parse(caminho_xml)
    root = tree.getroot()
    
    # Definir o namespace
    ns = {'nfe': 'http://www.portalfiscal.inf.br/cte'}
    
    # Função auxiliar para extrair texto de elementos com verificação
    def get_element_text(path):
        element = root.find(path, ns)
        return element.text if element is not None else ''
    
    # Ajuste para remover os dois primeiros dígitos
    def ajustar_codigo_municipio(codigo):
        return codigo[2:] if len(codigo) > 2 else codigo

    # Extrair as informações desejadas
    cte_info = {
        'CNPJ FILIAL': get_element_text('.//nfe:dest/nfe:CNPJ'),
        'CNPJ TRANSP.': get_element_text('.//nfe:emit/nfe:CNPJ'),
        'TRANSPORTADOR': get_element_text('.//nfe:emit/nfe:xNome'),
        'REMETENTE': get_element_text('.//nfe:rem/nfe:xNome'),
        'CNPJ_REMETENTE': get_element_text('.//nfe:rem/nfe:CNPJ'),
        'CTE': get_element_text('.//nfe:ide/nfe:nCT'),
        'SERIE': get_element_text('.//nfe:ide/nfe:serie'),
        'VALOR': get_element_text('.//nfe:vPrest/nfe:vTPrest').replace('.', ','),
        'CHAVEnf': '\n'.join([chave.text for chave in root.findall('.//nfe:infDoc/nfe:infNFe/nfe:chave', ns)]),
        'EMISSAO': get_element_text('.//nfe:ide/nfe:dhEmi').split('T')[0] if get_element_text('.//nfe:ide/nfe:dhEmi') else '',  # Formato de data apenas
        'UFIni': get_element_text('.//nfe:ide/nfe:UFIni'),
        'cMunIni': ajustar_codigo_municipio(get_element_text('.//nfe:ide/nfe:cMunIni')),
        'xMunIni': get_element_text('.//nfe:ide/nfe:xMunIni'),
        'UFFim': get_element_text('.//nfe:ide/nfe:UFFim'),
        'cMunFim': ajustar_codigo_municipio(get_element_text('.//nfe:ide/nfe:cMunFim')),
        'xMunFim': get_element_text('.//nfe:ide/nfe:xMunFim'),
        'chCTe': get_element_text('.//nfe:infProt/nfe:chCTe'),
        'vTrib': get_element_text('.//nfe:vPrest/nfe:vTPrest').replace('.', ','),
        'icms%': get_element_text('.//nfe:ICMS/nfe:ICMS00/nfe:pICMS'),
        'vICMS': get_element_text('.//nfe:ICMS/nfe:ICMS00/nfe:vICMS').replace('.', ','),
        'ICMSOutUF%': get_element_text('.//nfe:ICMS/nfe:ICMSOutraUF/nfe:pICMSOutraUF'),
        'vICMSOutUF': get_element_text('.//nfe:ICMS/nfe:ICMSOutraUF/nfe:vICMSOutraUF').replace('.', ',')
    }

    # Ajuste para a data de emissão no formato desejado (dd/mm/aaaa)
    if cte_info['EMISSAO']:
        data_emissao = cte_info['EMISSAO'].split('-')
        cte_info['EMISSAO'] = f"{data_emissao[2]}/{data_emissao[1]}/{data_emissao[0]}"
    
    return cte_info

# Percorrer todos os arquivos XML na pasta
for arquivo in os.listdir(caminho_pasta):
    if arquivo.endswith('.xml'):
        caminho_xml = os.path.join(caminho_pasta, arquivo)
        cte_info = extrair_dados_do_xml(caminho_xml)
        
        # Adicionar os valores fixos e inseridos via prompt
        cte_info['Venc.'] = vencimento
        cte_info['serviço'] = "FOB"
        cte_info['NOME FILIAL'] = f"CM {nome_filial_numero}"
        cte_info['COD'] = "000003"
        cte_info['Data lanç.'] = ""  # Deixa a coluna vazia para preenchimento manual posteriormente
        cte_info['cod_Rem']= r"PROCV([@[CNPJ_REMETENTE]];'C:\Users\dino.grejo\OneDrive - VIVEO\Documentos\!FOB CM HL\[cod_fornecedores.xlsx]fornecedores'!$B$1:$D$420;2;FALSO" #cod procV excel - extrair informação da planilha cod_fornecedores
        cte_info['loj_Rem']= r"PROCV([@[CNPJ_REMETENTE]];'C:\Users\dino.grejo\OneDrive - VIVEO\Documentos\!FOB CM HL\[cod_fornecedores.xlsx]fornecedores'!$B$1:$D$420;3;FALSO" #cod procV excel - extrair informação da planilha cod_fornecedores
        
        # Adicionar o dicionário de dados à lista
        dados.append(cte_info)

# Criar um DataFrame com os dados extraídos
df = pd.DataFrame(dados)

# Ajustar a ordem das colunas de acordo com o modelo Excel
df = df[['Venc.', 'Data lanç.', 'serviço', 'CNPJ FILIAL', 'NOME FILIAL', 'CNPJ TRANSP.', 'COD', 'TRANSPORTADOR', 
         'REMETENTE', 'CNPJ_REMETENTE','cod_Rem','loj_Rem','CTE', 'SERIE', 'VALOR', 'CHAVEnf', 'EMISSAO', 'UFIni', 'cMunIni', 
         'xMunIni', 'UFFim', 'cMunFim', 'xMunFim', 'chCTe', 'vTrib', 'icms%', 'vICMS', 'ICMSOutUF%', 'vICMSOutUF']]

# Gerar a planilha Excel
caminho_excel = r'C:\Users\dino.grejo\OneDrive - VIVEO\Documentos\python\Fatura_CTes_FOB.xlsx'
df.to_excel(caminho_excel, index=False)

print(f'Planilha gerada em: {caminho_excel}')


### **CODE AJUSTADO FOB HL - Fornecedor via .db**
#### **INSERIR VIA PROMPR VENCIMENTO E FILIAL**

In [4]:
# SCRIPT IN PYTHON BY DINO GREJO - DINOGREJO@GMAIL.COM GITHUB: DGREJ

import os
import xml.etree.ElementTree as ET
import pandas as pd
import sqlite3

# Pasta contendo os arquivos XML
caminho_pasta = r'C:\Users\dino.grejo\OneDrive - VIVEO\Documentos\python\xml'

# Solicitar valores via prompt
vencimento = input("Digite a data de vencimento para a coluna 'Venc.': ")
nome_filial_numero = input("Digite o número para a 'NOME FILIAL' (ex: 001002): ")

# Conectar ao banco de dados SQLite
conn = sqlite3.connect('fornecedores.db')
cursor = conn.cursor()

# Lista para armazenar os dados extraídos
dados = []

# Função para extrair as informações de cada XML
def extrair_dados_do_xml(caminho_xml):
    tree = ET.parse(caminho_xml)
    root = tree.getroot()
    
    # Definir o namespace
    ns = {'nfe': 'http://www.portalfiscal.inf.br/cte'}
    
    # Função auxiliar para extrair texto de elementos com verificação
    def get_element_text(path):
        element = root.find(path, ns)
        return element.text if element is not None else ''
    
    # Ajuste para remover os dois primeiros dígitos
    def ajustar_codigo_municipio(codigo):
        return codigo[2:] if len(codigo) > 2 else codigo

    # Extrair as informações desejadas
    cte_info = {
        'CNPJ FILIAL': get_element_text('.//nfe:dest/nfe:CNPJ'),
        'CNPJ TRANSP.': get_element_text('.//nfe:emit/nfe:CNPJ'),
        'TRANSPORTADOR': get_element_text('.//nfe:emit/nfe:xNome'),
        'REMETENTE': get_element_text('.//nfe:rem/nfe:xNome'),
        'CNPJ_REMETENTE': get_element_text('.//nfe:rem/nfe:CNPJ'),
        'CTE': get_element_text('.//nfe:ide/nfe:nCT'),
        'SERIE': get_element_text('.//nfe:ide/nfe:serie'),
        'VALOR': get_element_text('.//nfe:vPrest/nfe:vTPrest').replace('.', ','),
        'CHAVEnf': '\n'.join([chave.text for chave in root.findall('.//nfe:infDoc/nfe:infNFe/nfe:chave', ns)]),
        'EMISSAO': get_element_text('.//nfe:ide/nfe:dhEmi').split('T')[0] if get_element_text('.//nfe:ide/nfe:dhEmi') else '',  # Formato de data apenas
        'UFIni': get_element_text('.//nfe:ide/nfe:UFIni'),
        'cMunIni': ajustar_codigo_municipio(get_element_text('.//nfe:ide/nfe:cMunIni')),
        'xMunIni': get_element_text('.//nfe:ide/nfe:xMunIni'),
        'UFFim': get_element_text('.//nfe:ide/nfe:UFFim'),
        'cMunFim': ajustar_codigo_municipio(get_element_text('.//nfe:ide/nfe:cMunFim')),
        'xMunFim': get_element_text('.//nfe:ide/nfe:xMunFim'),
        'chCTe': get_element_text('.//nfe:infProt/nfe:chCTe'),
        'vTrib': get_element_text('.//nfe:vPrest/nfe:vTPrest').replace('.', ','),
        'icms%': get_element_text('.//nfe:ICMS/nfe:ICMS00/nfe:pICMS'),
        'vICMS': get_element_text('.//nfe:ICMS/nfe:ICMS00/nfe:vICMS').replace('.', ','),
        'ICMSOutUF%': get_element_text('.//nfe:ICMS/nfe:ICMSOutraUF/nfe:pICMSOutraUF'),
        'vICMSOutUF': get_element_text('.//nfe:ICMS/nfe:ICMSOutraUF/nfe:vICMSOutraUF').replace('.', ',')
    }

    # Ajuste para a data de emissão no formato desejado (dd/mm/aaaa)
    if cte_info['EMISSAO']:
        data_emissao = cte_info['EMISSAO'].split('-')
        cte_info['EMISSAO'] = f"{data_emissao[2]}/{data_emissao[1]}/{data_emissao[0]}"
    
    return cte_info

# Percorrer todos os arquivos XML na pasta
for arquivo in os.listdir(caminho_pasta):
    if arquivo.endswith('.xml'):
        caminho_xml = os.path.join(caminho_pasta, arquivo)
        cte_info = extrair_dados_do_xml(caminho_xml)
        
        # Buscar cod_Rem e loj_Rem no banco de dados usando CNPJ_REMETENTE
        cnpj_remetente = cte_info['CNPJ_REMETENTE']
        cursor.execute("SELECT cod, loja FROM fornecedores WHERE CNPJ_REMETENTE = ?", (cnpj_remetente,))
        resultado = cursor.fetchone()
        
        # Adicionar os valores de cod_Rem e loj_Rem ao dicionário
        if resultado:
            cte_info['cod_Rem'] = resultado[0]  # cod
            cte_info['loj_Rem'] = resultado[1]  # loja
        else:
            cte_info['cod_Rem'] = ''  # Valor vazio se não encontrar
            cte_info['loj_Rem'] = ''  # Valor vazio se não encontrar
        
        # Adicionar os valores fixos e inseridos via prompt
        cte_info['Venc.'] = vencimento
        cte_info['serviço'] = "FOB"
        cte_info['NOME FILIAL'] = f"CM {nome_filial_numero}"
        cte_info['COD'] = "000003"
        cte_info['Data lanç.'] = ""  # Deixa a coluna vazia para preenchimento manual posteriormente
        
        # Adicionar o dicionário de dados à lista
        dados.append(cte_info)

# Criar um DataFrame com os dados extraídos
df = pd.DataFrame(dados)

# Ajustar a ordem das colunas de acordo com o modelo Excel
df = df[['Venc.', 'Data lanç.', 'serviço', 'CNPJ FILIAL', 'NOME FILIAL', 'CNPJ TRANSP.', 'COD', 'TRANSPORTADOR', 
         'REMETENTE', 'CNPJ_REMETENTE','cod_Rem','loj_Rem','CTE', 'SERIE', 'VALOR', 'CHAVEnf', 'EMISSAO', 'UFIni', 'cMunIni', 
         'xMunIni', 'UFFim', 'cMunFim', 'xMunFim', 'chCTe', 'vTrib', 'icms%', 'vICMS', 'ICMSOutUF%', 'vICMSOutUF']]

# Gerar a planilha Excel
caminho_excel = r'C:\Users\dino.grejo\OneDrive - VIVEO\Documentos\python\Fatura_CTes_FOB.xlsx'
df.to_excel(caminho_excel, index=False)

# Fechar a conexão com o banco de dados
conn.close()

print(f'Planilha gerada em: {caminho_excel}')

Digite a data de vencimento para a coluna 'Venc.':  14/03/2025
Digite o número para a 'NOME FILIAL' (ex: 001002):  001029


Planilha gerada em: C:\Users\dino.grejo\OneDrive - VIVEO\Documentos\python\Fatura_CTes_FOB.xlsx


### **Carregar os dados do Excel**

Use o Pandas para ler o arquivo Excel:

In [ ]:
import pandas as pd
import sqlite3

# Passo 1: Carregar os dados do Excel
caminho_excel = r"C:\Users\dino.grejo\OneDrive - VIVEO\Documentos\python\cod_fornecedores.xlsx"  # Substitua pelo caminho do seu arquivo

# Ler o arquivo Excel, forçando as colunas CNPJ_REMETENTE, cod e loja como texto
df = pd.read_excel(caminho_excel, dtype={
    'CNPJ_REMETENTE': str,  # Forçar como texto
    'cod': str,             # Forçar como texto
    'loja': str             # Forçar como texto
})

# Substituir 'nan' por strings vazias ('') nas colunas cod e loja
df['cod'] = df['cod'].replace('nan', '')
df['loja'] = df['loja'].replace('nan', '')

# Exibir as primeiras linhas para verificar os dados
print(df.head())

### **Conectar ao banco de dados SQLite**
 
O SQLite é uma biblioteca embutida no Python, então você pode usá-la diretamente. Vamos criar um banco de dados e uma tabela para armazenar os dados do Excel.

```python
import sqlite3

# Conectar ao banco de dados (ou criar se não existir)
conn = sqlite3.connect('fornecedores.db')

# Criar um cursor para executar comandos SQL
cursor = conn.cursor()

In [ ]:
import sqlite3

# Conectar ao banco de dados (ou criar se não existir)
conn = sqlite3.connect('fornecedores.db')

# Criar um cursor para executar comandos SQL
cursor = conn.cursor()

### **Criar a tabela no banco de dados**

Agora, vamos criar uma tabela no banco de dados SQLite. O nome da tabela será fornecedores, e as colunas serão baseadas nas colunas do DataFrame do Pandas.

Substitua nome, idade, cidade, etc., pelos nomes das colunas do seu arquivo Excel.
```python
# Nome da tabela
nome_tabela = "fornecedores"

# Criar a tabela com os tipos corretos
cursor.execute(f'''
CREATE TABLE IF NOT EXISTS {nome_tabela} (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    REMETENTE TEXT,
    CNPJ_REMETENTE TEXT,  -- Definido como TEXT para preservar zeros à esquerda
    COD TEXT,             -- Definido como TEXT para preservar zeros à esquerda
    LOJA TEXT             -- Definido como TEXT para preservar zeros à esquerda
);
''')

# Confirmar a criação da tabela
conn.commit()

In [ ]:
# Nome da tabela
nome_tabela = "fornecedores"

# Criar a tabela com os tipos corretos
cursor.execute(f'''
CREATE TABLE IF NOT EXISTS {nome_tabela} (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    REMETENTE TEXT,
    CNPJ_REMETENTE TEXT,  -- Definido como TEXT para preservar zeros à esquerda
    COD TEXT,             -- Definido como TEXT para preservar zeros à esquerda
    LOJA TEXT             -- Definido como TEXT para preservar zeros à esquerda
);
''')

# Confirmar a criação da tabela
conn.commit()

### **Inserir os dados do Excel no banco de dados**
 
Agora, vamos inserir os dados do DataFrame (que veio do Excel) na tabela do banco de dados SQLite.

```python
# Inserir os dados na tabela
df.to_sql(nome_tabela, conn, if_exists='replace', index=False)

print(f"Dados exportados para a tabela '{nome_tabela}' no banco de dados!")



In [ ]:
# Inserir os dados na tabela
df.to_sql(nome_tabela, conn, if_exists='replace', index=False)

print(f"Dados exportados para a tabela '{nome_tabela}' no banco de dados!")

### **Consultar os dados no banco de dados**

Agora que os dados estão no banco de dados, você pode consultá-los usando o Pandas ou o SQLite diretamente.

### **Usando o Pandas:**

```python
# Consultar os dados da tabela
query = "SELECT * FROM fornecedores"
df_consulta = pd.read_sql(query, conn)

# Exibir os dados
df_consulta.head()

In [ ]:
# Consultar os dados da tabela
query = "SELECT * FROM fornecedores"
df_consulta = pd.read_sql(query, conn)

# Exibir os dados
df_consulta.head()

### **Usando o SQLite diretamente:**

```python
# Executar uma consulta SQL
cursor.execute("SELECT * FROM fornecedores")

# Recuperar os resultados
resultados = cursor.fetchall()

# Exibir os resultados
for linha in resultados:
    print(linha)

In [ ]:
# Executar uma consulta SQL
cursor.execute("SELECT * FROM fornecedores")

# Recuperar os resultados
resultados = cursor.fetchall()

# Exibir os resultados
for linha in resultados:
    print(linha)

### **Fechar a conexão com o banco de dados**

Quando terminar de trabalhar com o banco de dados, não se esqueça de fechar a conexão:

```python
conn.close()

In [ ]:
conn.close()

### **Conectar ao banco de dados**

Primeiro, precisamos conectar ao banco de dados SQLite usando o `sqlite3` (biblioteca embutida no Python) ou o `sqlalchemy` (recomendado para integração com o Pandas).

In [2]:
import sqlite3
import pandas as pd

# Conectar ao banco de dados (substitua 'fornecedores.db' pelo caminho do seu arquivo .db)
conn = sqlite3.connect('fornecedores.db')

# Criar um cursor para executar comandos SQL
cursor = conn.cursor()

### **Consultar os dados**

Você pode consultar os dados diretamente usando SQL ou carregar os dados em um DataFrame do Pandas para manipulação.

### **Consultar usando SQL**

```python
# Consultar todos os dados da tabela fornecedores
cursor.execute("SELECT * FROM fornecedores")
resultados = cursor.fetchall()

# Exibir os resultados
for linha in resultados:
    print(linha)

In [5]:
# Consultar todos os dados da tabela fornecedores
cursor.execute("SELECT * FROM fornecedores WHERE cod = '006578'") #Alterar o codigo de consulta SQL
resultados = cursor.fetchall()

# Exibir os resultados
for linha in resultados:
    print(linha)

('RENYLAB - QUIMICA E FARMACEUTICA LTDA', '00562583000144', '006578', '0001')


### **Consultar usando Pandas**

```python
# Carregar os dados da tabela fornecedores em um DataFrame
query = "SELECT * FROM fornecedores"
df = pd.read_sql(query, conn)

# Exibir as primeiras linhas do DataFrame
print(df.head())

In [7]:
# Carregar os dados da tabela fornecedores em um DataFrame
query = "SELECT * FROM fornecedores"
df = pd.read_sql(query, conn)

# Exibir as primeiras linhas do DataFrame
print(df.head())

                                       REMETENTE  CNPJ_REMETENTE     cod  loja
0  DIAGNOSTEK IND E COMERC PROD CIENTIFICOS LTDA  06223055000147    None  None
1         LABOFARMA PRODUTOS FARMACEUTICOS LTDA.  07863523001778  036584  0017
2             ACHE LABORATORIOS FARMACEUTICOS SA  60659463000191  002218  0001
3             CRAL ARTIGOS PARA LABORATORIO LTDA  48740849000713  015659  0007
4                          BLAU FARMACEUTICA S.A  58430828000160  120099  0001


### **Alterar os dados**

Você pode alterar os dados no banco de dados usando comandos SQL ou manipulando o DataFrame e enviando as alterações de volta para o banco de dados.

### **Alterar usando SQL**

```python
# Atualizar um registro na tabela fornecedores
cursor.execute("UPDATE fornecedores SET loja = '0002' WHERE cod = '02218'")

# Confirmar a alteração
conn.commit()

# Verificar a alteração
cursor.execute("SELECT * FROM fornecedores WHERE cod = '02218'")
print(cursor.fetchall())

In [ ]:
# Atualizar um registro na tabela fornecedores
cursor.execute("UPDATE fornecedores SET loja = '0002' WHERE cod = '02218'")

# Confirmar a alteração
conn.commit()

# Verificar a alteração
cursor.execute("SELECT * FROM fornecedores WHERE cod = '02218'")
print(cursor.fetchall())

### **Alterar usando Pandas**

```python
# Carregar os dados da tabela fornecedores em um DataFrame
df = pd.read_sql("SELECT * FROM fornecedores", conn)

# Alterar um valor no DataFrame
df.loc[df['cod'] == '02218', 'loja'] = '0002'

# Enviar as alterações de volta para o banco de dados
df.to_sql('fornecedores', conn, if_exists='replace', index=False)

# Verificar a alteração
print(df[df['cod'] == '02218'])

In [ ]:
# Carregar os dados da tabela fornecedores em um DataFrame
query = "SELECT * FROM fornecedores"
df = pd.read_sql(query, conn)

# Alterar um valor no DataFrame
df.loc[df['cod'] == '02218', 'loja'] = '0002'

# Enviar as alterações de volta para o banco de dados
df.to_sql('fornecedores', conn, if_exists='replace', index=False)

# Verificar a alteração
print(df[df['cod'] == '02218'])

### **Inserir novos dados**

Você pode inserir novos dados diretamente usando SQL ou adicionar linhas ao DataFrame e enviá-las para o banco de dados.

### **Inserir usando SQL**

```python
# Inserir um novo registro na tabela fornecedores
cursor.execute("INSERT INTO fornecedores (REMETENTE, CNPJ_REMETENTE, cod, loja) VALUES ('Novo Fornecedor', '12345678901234', '99999', '0001')")

# Confirmar a inserção
conn.commit()

# Verificar a inserção
cursor.execute("SELECT * FROM fornecedores WHERE cod = '99999'")
print(cursor.fetchall())

In [3]:
# Inserir um novo registro na tabela fornecedores
cursor.execute("INSERT INTO fornecedores (REMETENTE, CNPJ_REMETENTE, cod, loja) VALUES ('DC SUPRIMENTOS COMERCIAL DISTR LTDA', '38486893000108', '017109', '0001')")

# Confirmar a inserção
conn.commit()

# Verificar a inserção
cursor.execute("SELECT * FROM fornecedores WHERE cod = '017109'")
print(cursor.fetchall())

[('DC SUPRIMENTOS COMERCIAL DISTR LTDA', '38486893000108', '017109', '0001')]


### **Inserir usando Pandas**

```python
# Criar um novo DataFrame com os dados a serem inseridos
novo_fornecedor = pd.DataFrame({
    'REMETENTE': ['Novo Fornecedor'],
    'CNPJ_REMETENTE': ['12345678901234'],
    'cod': ['99999'],
    'loja': ['0001']
})

# Adicionar os novos dados ao DataFrame existente
df = pd.read_sql("SELECT * FROM fornecedores", conn)
df = pd.concat([df, novo_fornecedor], ignore_index=True)

# Enviar os dados atualizados de volta para o banco de dados
df.to_sql('fornecedores', conn, if_exists='replace', index=False)

# Verificar a inserção
print(df[df['cod'] == '99999'])

In [3]:
# Criar um novo DataFrame com os dados a serem inseridos
novo_fornecedor = pd.DataFrame({
    'REMETENTE': ['RENYLAB - QUIMICA E FARMACEUTICA LTDA'],
    'CNPJ_REMETENTE': ['00562583000144'],
    'cod': ['006578'],
    'loja': ['0001']
})

# Adicionar os novos dados ao DataFrame existente
df = pd.read_sql("SELECT * FROM fornecedores", conn)
df = pd.concat([df, novo_fornecedor], ignore_index=True)

# Enviar os dados atualizados de volta para o banco de dados
df.to_sql('fornecedores', conn, if_exists='replace', index=False)

# Verificar a inserção
print(df[df['cod'] == '006578'])

                                REMETENTE  CNPJ_REMETENTE     cod  loja
52  RENYLAB - QUIMICA E FARMACEUTICA LTDA  00562583000144  006578  0001


### **Excluir dados**

Você pode excluir dados diretamente usando SQL ou filtrar o DataFrame e enviar as alterações de volta para o banco de dados.

### **Excluir usando SQL**

```python
# Excluir um registro da tabela fornecedores
cursor.execute("DELETE FROM fornecedores WHERE cod = '99999'")

# Confirmar a exclusão
conn.commit()

# Verificar a exclusão
cursor.execute("SELECT * FROM fornecedores WHERE cod = '99999'")
print(cursor.fetchall())

In [ ]:
# Excluir um registro da tabela fornecedores
cursor.execute("DELETE FROM fornecedores WHERE cod = '99999'")

# Confirmar a exclusão
conn.commit()

# Verificar a exclusão
cursor.execute("SELECT * FROM fornecedores WHERE cod = '99999'")
print(cursor.fetchall())

### **Excluir usando Pandas**

```python
# Carregar os dados da tabela fornecedores em um DataFrame
df = pd.read_sql("SELECT * FROM fornecedores", conn)

# Filtrar os dados para excluir um registro
df = df[df['cod'] != '99999']

# Enviar os dados atualizados de volta para o banco de dados
df.to_sql('fornecedores', conn, if_exists='replace', index=False)

# Verificar a exclusão
print(df[df['cod'] == '99999'])

In [ ]:
# Carregar os dados da tabela fornecedores em um DataFrame
df = pd.read_sql("SELECT * FROM fornecedores", conn)

# Filtrar os dados para excluir um registro
df = df[df['cod'] != '99999']

# Enviar os dados atualizados de volta para o banco de dados
df.to_sql('fornecedores', conn, if_exists='replace', index=False)

# Verificar a exclusão
print(df[df['cod'] == '99999'])

### **Fechar a conexão**

Sempre feche a conexão com o banco de dados quando terminar de usá-lo:

```python
conn.close()

In [ ]:
conn.close()

### **Resumo:**

- **Consultar dados:** Use `cursor.execute()` para SQL ou `pd.read_sql()` para Pandas.
- **Alterar dados:** Use `UPDATE` em SQL ou manipule o DataFrame e use `to_sql()`.
- **Inserir dados:** Use `INSERT` em SQL ou adicione linhas ao DataFrame e use `to_sql()`.
- **Excluir dados:** Use `DELETE` em SQL ou filtre o DataFrame e use `to_sql()`.